# Probabilistic Bayesian Neural Networks

<table align="left">
  <td>
    <a 
    href="https://colab.research.google.com/github/gbih/machine_learning/blob/main/tf_probability/bayesian_neural_networks_wine.ipynb" 
    target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

**Annotation Notes (2022/08/23 George Baptista george@omame.com)**

This is an excellent notebook by Khalid Salama that compares the standard non-probabilistic NN model with a simple Bayesian NN, and ultimately a probabilistic Bayesian NN model.

* Experiment 1: standard neural network
* Experiment 2: Bayesian neural network (BNN)
* Experiment 3: probabilistic Bayesian neural network

The output of Experiment 1 (standard model) and Experiment 2 (BNN) are deterministic. This produces a point estimate as a prediction for a given example. 

The output of Experiment 3 is a probabilistic NN where the model outputs a probability distribution. Here, the model captures the aleatoric uncertainty as well, which is due to the irreducible noise in the data, or to the stochastic nature of the process generating the data.

In Experiment 3, we model the output as an IndependentNormal distribution, with learnable mean and variance parameters. 

If the task was classification, we would use IndependentBernoulli with binary classes, and OneHotCategorical with multiple classes to model distribution of the model output.

---

**Author:** [Khalid Salama](https://www.linkedin.com/in/khalid-salama-24403144/)<br>
**Date created:** 2021/01/15<br>
**Last modified:** 2021/01/15<br>
**Description:** Building probabilistic Bayesian neural network models with TensorFlow Probability.

## Introduction

Taking a probabilistic approach to deep learning allows to account for *uncertainty*,
so that models can assign less levels of confidence to incorrect predictions.
Sources of uncertainty can be found in the data, due to measurement error or
noise in the labels, or the model, due to insufficient data availability for
the model to learn effectively.


This example demonstrates how to build basic probabilistic Bayesian neural networks
to account for these two types of uncertainty.
We use [TensorFlow Probability](https://www.tensorflow.org/probability) library,
which is compatible with Keras API.

This example requires TensorFlow 2.3 or higher.

## The dataset

We use the [Wine Quality](https://archive.ics.uci.edu/ml/datasets/wine+quality)
dataset, which is available in the [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/wine_quality).
We use the red wine subset, which contains 4,898 examples.
The dataset has 11 numerical physicochemical features of the wine, and the task
is to predict the wine quality, which is a score between 0 and 10.
In this example, we treat this as a regression task.

## Setup

In [1]:
import sys

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Running on Colab. Installing tensorflow-datasets and tensorflow_probability.")
    !pip install tensorflow-probability &> /dev/null
    !pip tensorflow-datasets &> /dev/null
else:
    print("Running locally.")

Running on Colab. Installing tensorflow-datasets and tensorflow_probability.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [3]:
def HR():
    print("-"*50)

## Create training and evaluation datasets

Here, we load the `wine_quality` dataset using `tfds.load()`, and we convert
the target feature to float. Then, we shuffle the dataset and split it into
training and test sets. We take the first `train_size` examples as the train
split, and the rest as the test split.

In [4]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset


## Compile, train, and evaluate the model

In [5]:
hidden_units = [8, 8]
learning_rate = 0.001

def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(
        train_dataset, 
        epochs=num_epochs, 
        validation_data=test_dataset,
        verbose=0, # turn on if need to debug
    )
    HR()
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

## Create model inputs

In [6]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]

def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

---
## Experiment 1: standard neural network

We create a standard deterministic neural network model as a baseline.

In [7]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

Let's split the wine dataset into training and test sets, with 85% and 15% of
the examples, respectively.

In [8]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Now let's train the baseline model. We use the `MeanSquaredError`
as the loss function.

In [9]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
--------------------------------------------------
Model training finished.
Train RMSE: 0.752
Evaluating model performance...
Test RMSE: 0.746


We take a sample from the test set use the model to obtain predictions for them.
Note that since the baseline model is deterministic, we get a single a
*point estimate* prediction for each test example, with no information about the
uncertainty of the model nor the prediction.

In [10]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 6.4 - Actual: 6.0
Predicted: 6.2 - Actual: 7.0
Predicted: 5.7 - Actual: 5.0
Predicted: 6.4 - Actual: 7.0
Predicted: 6.5 - Actual: 7.0
Predicted: 5.8 - Actual: 5.0
Predicted: 6.0 - Actual: 6.0
Predicted: 6.6 - Actual: 7.0
Predicted: 6.3 - Actual: 4.0
Predicted: 5.5 - Actual: 6.0


---
## Experiment 2: Bayesian neural network (BNN)

The object of the Bayesian approach for modeling neural networks is to capture
the **epistemic uncertainty**, which is uncertainty about the model fitness,
due to limited training data.

The idea is that, instead of learning specific weight (and bias) *values* in the
neural network, the Bayesian approach learns **weight distributions**, from which we can sample to produce an output for a given input,
to encode weight uncertainty.

Thus, we need to define **prior** and the **posterior distributions** of these weights,
and the training process is to learn the parameters of these distributions.

In [11]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model

# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

We use the `tfp.layers.DenseVariational` layer instead of the standard
`keras.layers.Dense` layer in the neural network model.

In [12]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

The epistemic uncertainty can be reduced as we increase the size of the
training data. That is, the more data the BNN model sees, the more it is certain
about its estimates for the weights (distribution parameters).
Let's test this behaviour by training the BNN model on a small subset of
the training set, and then on the full training set, to compare the output variances.

### Train BNN  with a small training subset.

In [13]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
--------------------------------------------------
Model training finished.
Train RMSE: 0.839
Evaluating model performance...
Test RMSE: 0.79


Since we have trained a BNN model, the model produces a different output each time
we call it with the same input, since each time a new set of weights are sampled
from the distributions to construct the network and produce an output.

The less certain the mode weights are, the more variability (wider range) we will
see in the outputs of the same inputs.

In [14]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )

compute_predictions(bnn_model_small)

Predictions mean: 6.3, min: 6.06, max: 6.44, range: 0.39 - Actual: 6.0
Predictions mean: 6.22, min: 5.72, max: 6.46, range: 0.74 - Actual: 7.0
Predictions mean: 5.73, min: 5.07, max: 6.13, range: 1.05 - Actual: 5.0
Predictions mean: 6.23, min: 5.78, max: 6.44, range: 0.65 - Actual: 7.0
Predictions mean: 6.35, min: 5.97, max: 6.48, range: 0.5 - Actual: 7.0
Predictions mean: 5.98, min: 5.26, max: 6.37, range: 1.11 - Actual: 5.0
Predictions mean: 5.95, min: 5.31, max: 6.29, range: 0.98 - Actual: 6.0
Predictions mean: 6.39, min: 6.13, max: 6.48, range: 0.36 - Actual: 7.0
Predictions mean: 6.29, min: 5.88, max: 6.44, range: 0.57 - Actual: 4.0
Predictions mean: 5.59, min: 4.88, max: 6.13, range: 1.25 - Actual: 6.0


### Train BNN  with the whole training set.

In [15]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
--------------------------------------------------
Model training finished.
Train RMSE: 0.765
Evaluating model performance...
Test RMSE: 0.777
Predictions mean: 6.4, min: 6.15, max: 6.56, range: 0.41 - Actual: 6.0
Predictions mean: 6.23, min: 5.84, max: 6.48, range: 0.64 - Actual: 7.0
Predictions mean: 5.71, min: 5.31, max: 6.1, range: 0.78 - Actual: 5.0
Predictions mean: 6.34, min: 6.01, max: 6.54, range: 0.53 - Actual: 7.0
Predictions mean: 6.55, min: 6.36, max: 6.65, range: 0.29 - Actual: 7.0
Predictions mean: 5.93, min: 5.5, max: 6.3, range: 0.8 - Actual: 5.0
Predictions mean: 5.96, min: 5.58, max: 6.38, range: 0.81 - Actual: 6.0
Predictions mean: 6.61, min: 6.43, max: 6.69, range: 0.26 - Actual: 7.0
Predictions mean: 6.45, min: 6.18, max: 6.61, range: 0.43 - Actual: 4.0
Predictions mean: 5.52, min: 5.19, max: 5.94, range: 0.75 - Actual: 6.0


Notice that the model trained with the full training dataset shows smaller range
(uncertainty) in the prediction values for the same inputs, compared to the model
trained with a subset of the training dataset.

---
## Experiment 3: probabilistic Bayesian neural network

So far, the output of the standard and the Bayesian NN models that we built is
**deterministic**, that is, produces a point estimate as a prediction for a given example.

We can create a probabilistic NN by letting the model output a distribution.
In this case, the model captures the **aleatoric uncertainty** as well,
which is due to irreducible noise in the data, or to the stochastic nature of the
process generating the data.

In this example, we model the output as a `IndependentNormal` distribution,
with learnable mean and variance parameters. 

If the task was classification,
we would have used `IndependentBernoulli` with binary classes, and `OneHotCategorical`
with multiple classes, to model distribution of the model output.

In [16]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

Since the output of the model is a distribution, rather than a point estimate,
we use the [negative loglikelihood](https://en.wikipedia.org/wiki/Likelihood_function)
as our loss function to compute how likely to see the true data (targets) from the
estimated distribution produced by the model.

In [17]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)

num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
--------------------------------------------------
Model training finished.
Train RMSE: 1.06
Evaluating model performance...
Test RMSE: 1.055


Now let's produce an output from the model given the test examples.
The output is now a distribution, and we can use its mean and variance
to compute the confidence intervals (CI) of the prediction.

In [18]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 6.47, stddev: 0.85, 95% CI: [8.13 - 4.8] - Actual: 6.0
Prediction mean: 6.25, stddev: 0.83, 95% CI: [7.88 - 4.63] - Actual: 7.0
Prediction mean: 5.9, stddev: 0.72, 95% CI: [7.31 - 4.49] - Actual: 5.0
Prediction mean: 6.44, stddev: 0.84, 95% CI: [8.08 - 4.79] - Actual: 7.0
Prediction mean: 6.6, stddev: 0.84, 95% CI: [8.24 - 4.95] - Actual: 7.0
Prediction mean: 6.27, stddev: 0.8, 95% CI: [7.83 - 4.7] - Actual: 5.0
Prediction mean: 6.17, stddev: 0.77, 95% CI: [7.67 - 4.66] - Actual: 6.0
Prediction mean: 6.65, stddev: 0.84, 95% CI: [8.29 - 5.01] - Actual: 7.0
Prediction mean: 6.45, stddev: 0.83, 95% CI: [8.09 - 4.82] - Actual: 4.0
Prediction mean: 5.59, stddev: 0.69, 95% CI: [6.93 - 4.24] - Actual: 6.0
